In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# Install dependencies (no wandb)
# ───────────────────────────────────────────────────────────────────────────────
!pip install -q transformers sentencepiece torch nltk accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 40.6 MB/s eta 0:00:00


In [ ]:
!pip install sacrebleu
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 10.1 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
from sacrebleu import corpus_bleu
from nltk.translate.meteor_score import meteor_score
from sacrebleu.metrics import CHRF, TER

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# Imports & NLTK downloads
# ───────────────────────────────────────────────────────────────────────────────
import random
import numpy as np
import torch
import nltk
import unicodedata
from datasets import load_dataset
from transformers import (
    MarianTokenizer, MarianMTModel,
    Seq2SeqTrainingArguments, Seq2SeqTrainer,
    DataCollatorForSeq2Seq, EarlyStoppingCallback
)
from torch.optim import AdamW
from sacrebleu.metrics import BLEU
from nltk.translate.meteor_score import meteor_score

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# Device & Seed for Reproducibility
# ───────────────────────────────────────────────────────────────────────────────
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Set all seeds
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

Using device: cuda


In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# Load & Preprocess Dataset
# ───────────────────────────────────────────────────────────────────────────────
def clean(text):
    text = unicodedata.normalize("NFKC", text)
    return " ".join(text.strip().split()).replace("\u200f","").replace("\u200e","")

def prep(batch):
    return {
        "ar": [clean(x["ar"]) for x in batch["translation"]],
        "en": [clean(x["en"]) for x in batch["translation"]]
    }

ds = load_dataset("opus100", "ar-en")
train = ds["train"].shuffle(42).select(range(500)).map(prep, batched=True, remove_columns=["translation"])
valid = ds["validation"].shuffle(42).select(range(100)).map(prep, batched=True, remove_columns=["translation"])
test = ds["test"].shuffle(42).select(range(100)).map(prep, batched=True, remove_columns=["translation"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'TypeError: Failed to fetch'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/214k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/99.3M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/979k [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# Model Configuration
# ───────────────────────────────────────────────────────────────────────────────
model_name = "Helsinki-NLP/opus-mt-ar-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name).to(device)

# Freeze first encoder layer
for param in model.model.encoder.layers[0].parameters():
    param.requires_grad = False

model.gradient_checkpointing_enable()

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# Tokenization
# ───────────────────────────────────────────────────────────────────────────────
def tokenize_fn(batch):
    inputs = tokenizer(
        batch["ar"],
        padding="longest",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    )
    labels = tokenizer(
        batch["en"],
        padding="longest",
        truncation=True,
        max_length=128,
        return_tensors="pt"
    ).input_ids
    labels[labels == tokenizer.pad_token_id] = -100
    return {
        "input_ids": inputs.input_ids,
        "attention_mask": inputs.attention_mask,
        "labels": labels
    }

train_tok = train.map(tokenize_fn, batched=True, remove_columns=["ar","en"])
valid_tok = valid.map(tokenize_fn, batched=True, remove_columns=["ar","en"])


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# Training Setup
# ───────────────────────────────────────────────────────────────────────────────
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, pad_to_multiple_of=8)
# Metrics
bleu = BLEU(effective_order=True)

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    if isinstance(preds, tuple):
        preds = preds[0]

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    dec_preds = tokenizer.batch_decode(preds, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    dec_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=True)

    # BLEU score
    bleu_score = bleu.corpus_score(dec_preds, [dec_labels]).score

    # METEOR score
    meteor_scores = [
        meteor_score([nltk.word_tokenize(ref)], nltk.word_tokenize(pred))
        for pred, ref in zip(dec_preds, dec_labels)
    ]

    # Exact match accuracy
    exact_matches = [int(pred.strip() == ref.strip()) for pred, ref in zip(dec_preds, dec_labels)]
    accuracy = np.mean(exact_matches) * 100  # Percentage

    return {
        "bleu": bleu_score,
        "meteor": np.mean(meteor_scores) * 100,
        "accuracy": accuracy
    }

    return {"bleu": bleu_score, "meteor": np.mean(meteor_scores)*100}
# Training arguments (WandB removed)
training_args = Seq2SeqTrainingArguments(
    output_dir="./optimized-model",
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    gradient_accumulation_steps=2,
    fp16=(device=="cuda"),
    predict_with_generate=True,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="bleu",
    greater_is_better=True,
    logging_steps=50,
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="none"  # Explicitly disabled WandB
)

# Optimizer
optimizer = AdamW(model.parameters(), lr=3e-5)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tok,
    eval_dataset=valid_tok,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    optimizers=(optimizer, None),
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=3, early_stopping_threshold=0.1)
    ]
)



<ipython-input-10-60a4f7b8a347>:65: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# Train & Evaluate
# ───────────────────────────────────────────────────────────────────────────────
print("🚀 Starting training...")
trainer.train()
trainer.save_model("best-ar-en-translator")



🚀 Starting training...


Epoch,Training Loss,Validation Loss,Bleu,Meteor,Accuracy
1,No log,3.469186,20.183297,55.868476,8.000000
2,3.923000,2.892178,29.577154,52.098226,4.000000
3,3.923000,2.720107,27.767297,52.083050,5.000000
4,2.630000,2.616925,21.273752,50.784929,5.000000
5,2.070000,2.566470,27.579115,50.994018,5.000000


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.encoder.embed_positions.weight', 'model.decoder.embed_tokens.weight', 'model.decoder.embed_positions.weight', 'lm_head.weight'].


In [ ]:
print("\n🧪 Evaluating on test set...")
test_tok = test.map(tokenize_fn, batched=True, remove_columns=["ar","en"])
test_results = trainer.evaluate(
    test_tok,
    num_beams=5,
    length_penalty=0.9,
    no_repeat_ngram_size=3
)
print("\n🔥 Final Test Metrics:", test_results)


🧪 Evaluating on test set...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]


🔥 Final Test Metrics: {'eval_loss': 2.988255023956299, 'eval_bleu': 38.972413660797244, 'eval_meteor': 56.01396153876374, 'eval_accuracy': 10.0, 'eval_runtime': 13.5819, 'eval_samples_per_second': 7.363, 'eval_steps_per_second': 0.957, 'epoch': 5.0}


In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# Translation Function
# ───────────────────────────────────────────────────────────────────────────────
def translate(text, num_beams=5, temperature=0.9):
    inputs = tokenizer(
        unicodedata.normalize("NFKC", text),
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128
    ).to(device)

    outputs = model.generate(
        **inputs,
        num_beams=num_beams,
        temperature=temperature,
        early_stopping=True,
        max_length=128,
        repetition_penalty=1.2,
        no_repeat_ngram_size=3
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Sample translations
print("\n🌍 Sample Translations:")
sample_texts = [
    "على الرغم من التحديات، ما زلنا نؤمن بأن المستقبل يحمل الكثير من الفرص.",
    "اللغة العربية غنية ومليئة بالالثقافة.",
    "الوقت لا ينتظر أحداً، لذلك علينا أن نستغله بحكمة."
]
for text in sample_texts:
    translation = translate(text)
    print(f"\n📌 Arabic: {text}\n🦅 English: {translation}")


🌍 Sample Translations:


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



📌 Arabic: على الرغم من التحديات، ما زلنا نؤمن بأن المستقبل يحمل الكثير من الفرص.
🦅 English: In spite of the challenges, we continue to believe that the future holds so many opportunities.

📌 Arabic: اللغة العربية غنية ومليئة بالالثقافة.
🦅 English: Arabic is rich and full of culture.

📌 Arabic: الوقت لا ينتظر أحداً، لذلك علينا أن نستغله بحكمة.
🦅 English: Time isn't waiting for no one, so we have to sage it.
